# 蘭花辨識

History
- 4/8 
    - v1
- 4/9
    - modify data generator - flow_from_dataframe
    - implement multiple model training
- 4/10
    - cutmix
    - ensemble
- 4/12
    - detect flower and cut images
- 4/13
    - refactor to class
    - pseudo labeling
    
TODO
  - pseudo labeling

In [1]:
%load_ext autoreload
%autoreload 2

import os, sys, cv2, glob, datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import tensorflow as tf
import time
import joblib
from PIL import Image
from colorama import Fore, Back, Style
from utils import *    
from trainner import *
from classifier import *
from stacking import *

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 


### 1. Prepare Data

In [2]:

data_floder = '/mnt/hdd1/Data/Competition/orchid/training'
df_tr = pd.read_csv(os.path.join(data_floder,'label.csv'))
df_tr['category_str'] = df_tr['category'].astype('str')
df_tr['filepath']=df_tr['filename'].map(lambda x: os.path.join(data_floder,x))
df_tr['key']=df_tr['filename']
df_tr['isAug']=False

#新增training_aug
aug_folder='/mnt/hdd1/Data/Competition/orchid/training_aug'
df_traug = pd.read_csv(os.path.join(aug_folder,'pseudo_label.csv'))
df_traug['category_str'] = df_traug['best_cls'].astype('str')
df_traug['category'] = df_traug['category_str'].astype(int)
df_traug['filepath']=df_traug['filename'].map(lambda x: os.path.join(aug_folder,x))
df_traug['key']=df_traug['filename'].map(lambda x: x[4:])
df_traug['isAug']=True

#TODO: 新增test_public

#TODO: 新增test_private

df_raw = pd.concat([df_tr])
df_aug = pd.concat([df_traug])

df = pd.concat([df_raw,df_aug])

### 2. Train Classification Model

In [3]:
agent = Trainner()
#agent.pretrain_dir = '/mnt/hdd1/Model/TF_Pretrain'
agent.pretrain_dir = '/mnt/hdd1/Model/Competition/orchid/0414v1'
#agent.pretrain_dir = None
agent.model_dir = '/mnt/hdd1/Model/Competition/orchid'
agent.cutmix=True
agent.tiral_ver = '0415v1'
agent.cls_number=219
agent.lr=1e-5 #0.1 
agent.opt='adamw'
agent.multi_gpu=False
EPOCH =50




In [ ]:
trtime_dict = {}
trsuccess_dict = {}
if agent.multi_gpu:
    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        for model_name in ensemble_models:
            df = shuffle(df)
            agent.training(model_name, df, EPOCH)
else:
    for model_name in ensemble_models:
        df = shuffle(df)
        ts = time.time()
        retry_cnt=3
        tr_success = False
        while True:       
            try:
                agent.training(model_name, df, EPOCH)
                print(Fore.GREEN + '---- PASS ----')
                print(Style.RESET_ALL)
                tr_success=True
                break
            except tf.errors.ResourceExhaustedError as e:
                print(Fore.RED +'---- ERROR ----', str(e))
                print(Style.RESET_ALL)
                if retry_cnt==0:
                    break            
                agent.reduce_batch(model_name, 4)    
                retry_cnt=retry_cnt-1
        
        t = time.time()-ts
        trtime_dict[model_name]=t
        trsuccess_dict[model_name]=tr_success
        print(f'{model_name}: {t:.2f} sec')
    

status_df = pd.DataFrame(list(trtime_dict.items()))
status_df['success'] = trsuccess_dict.values()
joblib.dump(status_df, f'model/trtime_{agent.tiral_ver}.pkl')


====== densenet201 ========
Found 3504 validated image filenames belonging to 219 classes.
Found 876 validated image filenames belonging to 219 classes.
Found 3504 validated image filenames belonging to 219 classes.
---- ERROR ---- failed to allocate memory [Op:AddV2]

densenet201 batch size: 26
====== densenet201 ========
Found 3504 validated image filenames belonging to 219 classes.
Found 876 validated image filenames belonging to 219 classes.
Found 3504 validated image filenames belonging to 219 classes.
---- ERROR ---- failed to allocate memory [Op:Mul]

densenet201 batch size: 22
====== densenet201 ========
Found 3504 validated image filenames belonging to 219 classes.
Found 876 validated image filenames belonging to 219 classes.
Found 3504 validated image filenames belonging to 219 classes.
---- ERROR ---- failed to allocate memory [Op:Mul]

densenet201 batch size: 18
====== densenet201 ========
Found 3504 validated image filenames belonging to 219 classes.
Found 876 validated im

In [ ]:
#training time
display(status_df)

#model loss/acc
model_dir = os.path.join(agent.model_dir, agent.tiral_ver)
for p in glob.glob(os.path.join(model_dir,'*.jpg')):
    img=Image.open(p)
    display(img)

### 3. Train Stacking Model

In [ ]:
clf = Classifier()
clf.model_dir = '/mnt/hdd1/Model/Competition/orchid'
clf.cls_map_path = 'model/model_cls_map.pkl'
clf.model_ver = agent.tiral_ver
clf.ensemble_models = ensemble_models


In [ ]:

df_raw, pred_list_raw = clf.inference(df_raw)
df_aug, pred_list_aug = clf.inference(df_aug)
pred_list = pred_list_raw+pred_list_aug
stackedX = stacked_dataset(pred_list)


stk_model = fit_stacked_model(stackedX, df_raw['category'])
yhat = stk_model.predict(stackedX)
stack_acc = accuracy_score(df_raw['category'], yhat)
print(f'Stack acc: {stack_acc:.2f}')

#Save stack model
joblib.dump(stk_model, os.path.join(agent.model_dir, agent.tiral_ver, 'stack.pkl'))
